# Исследование надежности заемщиков

**Задача**
<br>
Выяснить, как на погашение клиентом банка кредита в срок влияют следующие факторы:
- Количество детей
- Семейное положение
- Уровень дохода
- Цель кредита

## Анализ информации о данных

In [1]:
# Импортируем библиотеки
import pandas as pd

In [2]:
# Создадим датасет
data = pd.read_csv('data.csv')

In [3]:
# Выведем первые 20 строчек
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
# Выведем основную информацию о датафрейме с помощью метода info().
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Обработка пропусков

In [5]:
# Выведем количество пропущенных значений
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Заполним пропуски в столбце `total_income` медианными значениями по каждого типа занятости `income_type`.

In [6]:
# перебираем каждый тип занятости в наборе уникальных значений столба income_type
for i in data['income_type'].unique():
    #print(i)
    # на каждом шаге цикла с помощью атрибута loc выбираем строки,
    # в которых в income_type текущий тип занятости (i) и есть пропуски (True) в total_income 
    data.loc[(data['income_type'] == i) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == i), 'total_income'].median()
    # и записываем в них среднее значение total_income среди строк с текущим типом занятости (i)

Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.

In [7]:
# Заполним пропуски в столбце days_employed медианными значениями по каждого типа занятости income_type.
for i in data['income_type'].unique():
    #print(i)
    # на каждом шаге цикла с помощью атрибута loc выбираем строки,
    # в которых в income_type текущий тип занятости (i) и есть пропуски (True) в total_income 
    data.loc[(data['income_type'] == i) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == i), 'days_employed'].median()
    # и записываем в них среднее значение total_income среди строк с текущим типом занятости (i)

In [8]:
# Проверим заполнение пропусков
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Обработка аномалий

В данных были замечены отрицательные значения в столбце `days_employed`. Явно ошибка в данных, т.к. стаж не может быть отрицательным. Заменим все отрицательные значения положительными с помощью метода `abs()`.

In [9]:
data['days_employed'] = data['days_employed'].abs()

In [10]:
# Для каждого типа занятости выведем медианное значение трудового стажа days_employed в днях.
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть. Тем более этот столбец не понадобится для исследования.

In [11]:
# Выведем перечень уникальных значений столбца children.
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

In [12]:
# В столбце children обнаружены два аномальных значения. Удалим строк с этими значениями
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [13]:
# Проверим
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

### Изменение типов данных

In [14]:
# Заменим вещественный тип данных в столбце total_income на целочисленный с помощью метода astype().
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

In [15]:
# Обработаем неявные дубликаты в столбце education. Приведите их к нижнему регистру.
data['education'] = data['education'].str.lower()

In [16]:
# Выведем количество строк-дубликатов в данных
data.duplicated().sum()

71

In [17]:
# Удалим дубликаты
data = data.drop_duplicates()

### Категоризация данных

Создадим в датафрейме `data` столбец `total_income_category` с категориями в засимости от значения столбца `total_income`:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`

In [18]:
# Создадим функцию для присвоения категории
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [19]:
# Создадим новый столбец с категориями дохода с помощью функции
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [20]:
# Выведем перечень уникальных целей взятия кредита из столбца purpose.
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`

In [21]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [22]:
# Присвоим категории с помощью функции
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Исследование данных и ответьте на вопросы

#### Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [23]:
# Сгрупприуем данные по колонке 'children' со значениями: число кредитов, число должников, доля должников
data.groupby('children')['debt'].agg(['count', 'sum', 'mean']).sort_values('mean')

,count,sum,mean
children,,,
5,9,0,0.000000
0,14091,1063,0.075438
3,330,27,0.081818
1,4808,444,0.092346
2,2052,194,0.094542
4,41,4,0.097561


**Вывод:**
<br>
Самая высокая вероятность возврата кредита в срок в группе "без детей", процент невозврата в срок здесь 7.5%<br>
В группе с 3-я детьми вероятность возврата чуть хуже и составляет 8.2%, но разница с группой "без детей" менее 1%.<br>
В группах с 1-им, 2-мя и 4-мя детьми вероятности возврата кредита в срок примерно равны и составляют 9.2%, 9.5% и 9.8% соответственно.<br>
Вероятно, в группе с 4-мя детьми, с увеличением количества выданных кредитов в этой группе, вероятность возврата в срок будет увеличиваться. В пользу этого может говорить уменьшение вероятности невозврата кредита в срок до 8,2% в группе с 3-я детьми в сравнении с показателем 9,2-9,5% групп с 1-им и 2-я детьми.<br>
Статистику по группу с количеством детей "5" считаю не информативной, т.к. количество выданных кредитов в этой группе всего 9. Считаю, что такого количества недостаточно, чтобы делать выводы.<br>
В итоге замтено возрастание риска невозврата кредита в срок на 2% в случае, если у клиента есть есть дети.

#### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [24]:
# Сгрупприуем данные по колонке 'family_status' со значениями: число кредитов, число должников, доля должников
data.groupby('family_status')['debt'].agg(['count', 'sum', 'mean']).sort_values('mean')

,count,sum,mean
family_status,,,
вдовец / вдова,951,63,0.066246
в разводе,1189,84,0.070648
женат / замужем,12261,927,0.075606
гражданский брак,4134,385,0.093130
Не женат / не замужем,2796,273,0.097639


**Вывод:**
<br>
Можно объединить группы "вдовец/вдова" и "в разводе", т.к. по сути они схожи и здесь можно считать вероятность невозврата кредита в срок 6,8%. В группе "женат / за мужем" вероятность невозврата кредита в срок увеличивается на 1% до 7,6%.<br> 
В случае если клиент "не женат / не замужем" или состоит в "гражданском браке", вероятность невозврата кредита увеличивается почти на 2% до 9,5% в сравнении с группой "женат / за мужем", где этот показатель составляет 7.6%.<br>
Можно ометить, что вероятность невозврата кредита в срок в группе "гражданский брак" - 9.3% сильно ближе к вероятности в группе "Не женат / не замужем" - 9.8%, чем к группе "женат / замужем", где этот показатель составляет 7.6%. Вероятно, это связано с тем, что клиент мог указать семейное положение как "Не женат / не замужем", по факту состоя в "гражданском браке". Поэтому можно считать показатель в этих группах как 9.5%.<br>
Можно сделать вывод, что если клиент "Не женат / не замужем" или состоит в "гражданском браке", вероятность невозврата кредита в срок увеличивается на 2% в сравнени с группой "женат / за мужем" и на 2,7% в сравнении с группами "вдовец/вдова" и "в разводе".<br> 
В итоге при сравнении с остальными группами, вероятность невозврата кредита в срок увеличивается на 2.3%, если клиент "Не женат / не замужем" или состоит в "гражданском браке".

#### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [25]:
# Сгрупприуем данные по колонке 'total_income_category' со значениями: число кредитов, число должников, доля должников
data.groupby('total_income_category')['debt'].agg(['count', 'sum', 'mean']).sort_values('mean')

,count,sum,mean
total_income_category,,,
D,349,21,0.060172
B,5014,354,0.070602
A,25,2,0.080000
C,15921,1353,0.084982
E,22,2,0.090909


**Вывод:**
<br>
Ститистику по группам "E" (с доходом менее 30 000 руб) "A" (с доходом "более 1 000 001 руб) считаю неинформативной, т.к. количество выданных кредитов в этих группах очень мало - 22 и 25 соответственно, чтобы делать выводы.<br>
Если анализировать остальные группы, то самая высокая вероятность возврата кредита в срок в группе "D" (с ежемесячным доходом 30 001 - 50 000 руб), процент невозвращённых в срок кредитов здесь составил 6%.<br>
В группе "C" (c ежемесячным доходом 50 001 - 200 000 руб) вероятность возврата в срок меньше на 2.5% процента, доля невозвращённых в срок кредитов здесь составила 8.5%.<br>
В следующей группе, "B" (с ежемесячным доходом 200 001 - 1 000 000 руб), вероятность невозврата кредита в срок уменьшается до 7.1%. Однако, этот показатель также выше 6% - показателя группы "D" (с ежемесячнм доходом 30 001 - 50 000 руб)<br>
Можно сделать вывод, что если доход клиента выше 50 000 руб., вероятность возврата кредита в срок уменьшается на 1.1 - 2.5%

#### Как разные цели кредита влияют на его возврат в срок?

In [26]:
# Создадим сводную таблицу с группировкйо по колонке 'purpose_category' со значениями: 
# число кредитов, число должников, доля должников
data.pivot_table(index=['purpose_category'], values='debt', aggfunc=['count', 'sum', 'mean']).sort_values(('mean', 'debt'))

,count,sum,mean
,debt,debt,debt
purpose_category,,,
операции с недвижимостью,10751,780,0.072551
проведение свадьбы,2313,183,0.079118
получение образования,3988,369,0.092528
операции с автомобилем,4279,400,0.093480


**Вывод:**<br>
Самая высокая вероятность возврата кредита в срок в группах с целями кредита  "операции с недвижимостью" и "проведение свадьбы", процент невозвращённых в срок редитов здесь составил 7,3% и 7,9% соответственно.<br>
Самая высокая вероятность невозврата кредита в срок в группах с целями кредита  "получение образования" и "операции с автомобилем", процент невозвращённых в срок кредитов здесь составил 9,3%.<br>
Можно сделать вывод, что если целью кредита является "получение образования" и "операции с автомобилем", то вероятность невозврата кредита в срок увеличивается на 1,4 - 2%.

### Общий вывод

**Зависимость между количеством детей и возвратом кредита в срок**<br>
Заметно возрастание риска невозврата кредита в срок на 2% в случае, если у клиента есть есть дети.
<br><br>
**Зависимость между семейным положением и возвратом кредита в срок**<br>
Вероятность невозврата кредита в срок увеличивается на 2.3%, если клиент "Не женат / не замужем" или состоит в "гражданском браке"
<br><br>
**Зависимость между уровнем дохода и возвратом кредита в срок**<br>
Если доход клиента выше 50 000 руб., вероятность возврата кредита в срок уменьшается на 1.1 - 2.5%
<br><br>
**Влияние цели кредита на его возврат в срок**<br>
Если целью кредита является "получение образования" и "операции с автомобилем", то вероятность невозврата кредита в срок увеличивается на 1,4 - 2%.